In [1]:
! pip install scikit-learn==1.4.0

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
import graphviz
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import pycaret
from pycaret.classification import load_model,predict_model
from pycaret.classification import ClassificationExperiment
import category_encoders as ce
import modelbit


  Using cached scikit_learn-1.4.0-1-cp39-cp39-macosx_12_0_arm64.whl (10.7 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.3.2 requires scikit-learn>1.4.0, but you have scikit-learn 1.4.0 which is incompatible.
You should consider upgrading via the '/Users/hassanraufbutt/.pyenv/versions/3.9.13/envs/leadScore/bin/python3.9 -m pip install --upgrade pip' command.


In [21]:
user_input = {
    "1-Abschluss": "Example1",
    "1-Abschluss in DE": "Example2",
    "1-Deutscher Ehepartner": "Example3",
    "1-EB/NE erfllt?": "Example4",
    "1-Einreisejahr": "Example5",
    "1-Antrag EB": "Example6",
    "1-Antrag NE": "Example7",
    "1-Integrationstest": "Example8",
    "1-Jahr AR beantragt/bekommen": "Example9",
    "1-Jobcenter": "Example10",
    "1-Kinder": 2,
    "1-Netto": "Example11",
    "1-Rente": "Example12",
    "1-Sprachzertifikat": "Example13",
    "1-Test Sprache": "Example14",
    "1-Welches befristete AR haben Sie?": "Example15",
    "1-Wie ist ihr aktueller Familienstand?": "Example16",
    "1-Beratung?": "Ja",
    "1-Gltiger Nationalpass": "Example17",
    "Id": "12345",
    "Zeitpunkt der Erstellung": "2023-01-01",
    "Sales": 1,
    "Zeitpunkt der nderung": "2023-01-02",
    "Zeitpunkt der Erstellung - Year": 2023,
    "Zeitpunkt der nderung - Year": 2023,
    "SalesCount": 1
}

In [23]:
def remove_non_ascii(text):
    return ''.join(char for char in text if ord(char) < 128)

In [27]:
def preprocess_data(self, df) -> bool:
        try:
            new_df = df[['Abschluss', 'Abschluss_in_DE', 'Deutscher_Ehepartner', 'EB_NE_erf_llt', 'Einreisejahr', 'Haben_Sie_bereits_einen_Antrag_auf_Einb_rgerung_ge', 'Haben_Sie_bereits_einen_Antrag_auf_ein_unbefristet', 'Integrationstest',
                         'Jahr_AR_beantragt_bekommen', 'Jobcenter', 'Kinder', 'Netto', 'Rente', 'Sprachzertifikat1', 'Test_Sprache', 'Welches_befristete_AR_haben_Sie', 'Wie_ist_ihr_aktueller_Familienstand', 'Beratung', 'G_ltiger_Nationalpass', 'id', 'Sales']]
            new_df.columns = ['1-Abschluss', '1-Abschluss in DE', '1-Deutscher Ehepartner', '1-EB/NE erfllt?', '1-Einreisejahr', '1-Antrag EB', '1-Antrag NE', '1-Integrationstest', '1-Jahr AR beantragt/bekommen', '1-Jobcenter',
                              '1-Kinder', '1-Netto', '1-Rente', '1-Sprachzertifikat', '1-Test Sprache', '1-Welches befristete AR haben Sie?', '1-Wie ist ihr aktueller Familienstand?', '1-Beratung?', '1-Gltiger Nationalpass', 'Id', 'Sales']
            temp_df = new_df
            temp_df["Sales"] = temp_df["Sales"].astype('int64')
            temp_df["SalesCount"] = temp_df["Sales"].apply(
                lambda x: 1 if x > 0 else 0)
            temp_df["SalesCount"] = temp_df["SalesCount"].astype('int64')
            temp_df = temp_df.dropna(subset=["Sales"])
            temp_df = temp_df[temp_df["1-Abschluss"] != ""]
            replace_values_dict = {
                "1-Abschluss in DE": {"": "Not Asked"},
                "1-Deutscher Ehepartner": {"": "Not Asked"},
                "1-Einreisejahr": {"Vor 2014": "Vor 2015", "2011 oder davor": "Vor 2015", "2012": "Vor 2015", "2013": "Vor 2015", "2014": "Vor 2015"},
                "1-Jahr AR beantragt/bekommen": {"Vor 2014": "Vor 2015", "2014": "Vor 2015", "": "Not Asked"},
                "1-Jobcenter": {"": "Not Asked"},
                "1-Rente": {"": "Not Asked"},
                "1-Sprachzertifikat": {"": "Not Asked"},
                "1-Test Sprache": {"": "Deutsch"}
            }
            temp_df.replace(replace_values_dict, inplace=True)
            temp_df = temp_df[temp_df["1-Antrag EB"] != ""]
            temp_df = temp_df[temp_df["1-Beratung?"].isin(["Ja", "Nein"])]
            temp_df = temp_df[temp_df["1-Welches befristete AR haben Sie?"] != ""]
            temp_df = temp_df.applymap(remove_non_ascii)
            return temp_df
        except Exception as e:
            raise e

In [26]:
def prepare_input_data(input_data):
    columns = [
        "1-Abschluss", "1-Abschluss in DE", "1-Deutscher Ehepartner", "1-EB/NE erfllt?", "1-Einreisejahr",
        "1-Antrag EB", "1-Antrag NE", "1-Integrationstest", "1-Jahr AR beantragt/bekommen", "1-Jobcenter",
        "1-Kinder", "1-Netto", "1-Rente", "1-Sprachzertifikat", "1-Test Sprache",
        "1-Welches befristete AR haben Sie?", "1-Wie ist ihr aktueller Familienstand?", "1-Beratung?",
        "1-Gltiger Nationalpass", "Id", "Zeitpunkt der Erstellung", "Sales", "Zeitpunkt der nderung",
        "Zeitpunkt der Erstellung - Year", "Zeitpunkt der nderung - Year", "SalesCount"
    ]

    # Create a DataFrame with the input data
    input_df = pd.DataFrame([input_data], columns=columns)

    return input_df

In [38]:
def get_prediction(input_data):
    input_df = preprocess_data(input_data)
    input_df.drop(columns=["Id","SalesCount","1-Beratung?","1-Welches befristete AR haben Sie?","1-Test Sprache","Zeitpunkt der Erstellung","Sales","Zeitpunkt der nderung","Zeitpunkt der Erstellung - Year","Zeitpunkt der nderung - Year"], inplace=True)
    predicted_class = finalAlpha.predict(input_df)
    probability = finalAlpha.predict_proba(input_df)
    response = {
        'predicted_class': predicted_class,
        'probability': probability
    }

    return response

In [40]:
def prediction(input_data):
    input_df = prepare_input_data(input_data)
    input_df.drop(columns=["Id","SalesCount","1-Beratung?","1-Welches befristete AR haben Sie?","1-Test Sprache","Zeitpunkt der Erstellung","Sales","Zeitpunkt der nderung","Zeitpunkt der Erstellung - Year","Zeitpunkt der nderung - Year"], inplace=True)
    predicted_class = finalAlpha.predict(input_df)
    probability = finalAlpha.predict_proba(input_df)
    response = {
        'predicted_class': predicted_class,
        'probability': probability
    }

    return response

In [29]:
md=modelbit.login()

In [30]:
df = pd.read_csv('/Users/hassanraufbutt/Desktop/migrando/Migrando_Internal_CodeOnly/data/shortCleanData.csv')
df
df_filtered = df[df['1-Beratung?'] != 'Nein']

In [31]:
classifyExp = ClassificationExperiment()
encoder = ce.GLMMEncoder()

In [32]:

train = classifyExp.setup(df_filtered,target = 'SalesCount',ignore_features = ["Id","1-Beratung?","1-Welches befristete AR haben Sie?","1-Test Sprache","Zeitpunkt der Erstellung","Sales","Zeitpunkt der nderung","Zeitpunkt der Erstellung - Year","Zeitpunkt der nderung - Year"],train_size=0.7,preprocess=True,max_encoding_ohe=1,encoding_method=encoder,normalize=True,normalize_method='minmax',feature_selection=True,feature_selection_method="univariate",n_features_to_select=0.25)
train.get_config("train")
init_model = train.compare_models(sort='F1',cross_validation=True,probability_threshold=0.075)
model = train.create_model('qda',cross_validation=False,probability_threshold=0.075)
dfCopy = df.copy()

,Description,Value
0,Session id,3358
1,Target,SalesCount
2,Target type,Binary
3,Original data shape,"(12366, 26)"
4,Transformed data shape,"(12366, 5)"
5,Transformed train set shape,"(8656, 5)"
6,Transformed test set shape,"(3710, 5)"
7,Ignore features,9
8,Numeric features,1
9,Categorical features,15


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.3808,0.6073,0.7941,0.1306,0.2243,0.0381,0.0835,1.0970
knn,K Neighbors Classifier,0.5747,0.5602,0.5422,0.1405,0.2229,0.0537,0.0773,1.0740
lr,Logistic Regression,0.2728,0.6121,0.9057,0.1248,0.2193,0.0263,0.0805,1.3910
rf,Random Forest Classifier,0.3701,0.5874,0.7818,0.1272,0.2188,0.0308,0.0683,1.3780
qda,Quadratic Discriminant Analysis,0.2825,0.6069,0.8883,0.1244,0.2182,0.0255,0.0754,1.2680
et,Extra Trees Classifier,0.3742,0.5860,0.7726,0.1268,0.2178,0.0299,0.0658,1.2980
lightgbm,Light Gradient Boosting Machine,0.3380,0.5963,0.8166,0.1256,0.2177,0.0276,0.0672,1.4820
dt,Decision Tree Classifier,0.3734,0.5839,0.7716,0.1265,0.2173,0.0293,0.0646,1.1520
lda,Linear Discriminant Analysis,0.2576,0.6100,0.9129,0.1232,0.2171,0.0230,0.0743,1.2410
gbc,Gradient Boosting Classifier,0.2818,0.6046,0.8812,0.1235,0.2166,0.0235,0.0701,1.2820


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Test,0.2709,0.5853,0.8995,0.1237,0.2175,0.0242,0.0744


In [33]:
train.predict_model(model,data=dfCopy)
finalAlpha = train.finalize_model(model)
md.add_model("leadScore",model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.2459,0.5987,0.9034,0.0667,0.1242,0.0157,0.0639


Uploading 'leadScore': 100%|██████████| 1.04k/1.04k [00:01<00:00, 945B/s]
Updating registry: 100%|██████████| 1/1 [00:01<00:00]


In [41]:
print(prediction(user_input))
md.deploy(get_prediction)

{'predicted_class': array([1]), 'probability': array([[0.88380429, 0.11619571]])}


In [42]:
md.deploy(prediction)